In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Self-Attention Implementation for Single Head

In [48]:
B,T,C = 4, 8, 32  
n_embd = 32
head_size = 16
x = torch.randn(B, T, C)
key = nn.Linear(n_embd, head_size, bias=False)
query = nn.Linear(n_embd, head_size, bias=False)
value = nn.Linear(n_embd, head_size, bias=False)
k = key(x)   # (B,T,hs)
q = query(x) # (B,T,hs)
tril = torch.tril((torch.ones(T, T)))
wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

In [49]:
k[0]    # key_shape = (8,32) for first batch

tensor([[ 0.1453,  0.0093, -0.6227,  0.6044, -0.5223, -0.0383, -0.0409,  0.0957,
          0.1706,  0.2848,  0.5654,  0.5045, -0.4644, -0.2814,  0.3140,  0.4868],
        [ 0.3928, -0.1117, -0.4124,  0.1512,  0.4923,  0.4879,  0.9946,  0.5830,
          0.4995, -0.4319, -0.3418,  1.1051, -0.2307, -0.0193,  0.4881, -1.0514],
        [-0.1744, -1.2138,  0.2995,  0.7032,  0.4439, -0.2391, -0.1909, -0.0995,
         -0.1237, -0.1534, -0.1126, -0.2357,  1.0341, -0.3448, -0.2266, -0.2628],
        [-0.3164, -0.2148,  0.9246, -0.1540,  1.1323, -0.7732,  0.0673,  0.1828,
         -0.4045,  0.3284,  0.2464,  0.3437, -0.3202,  0.2157,  1.1788,  0.1674],
        [-0.1141, -0.7778, -1.2740,  0.9140, -0.2167, -0.4946,  0.2256,  0.9484,
          0.4386, -0.3848,  0.3216, -0.3488, -0.2130, -0.0883, -0.4912,  0.3360],
        [ 1.0447,  0.0186, -0.5613,  0.1867,  0.5278,  0.4136,  0.0867,  0.2016,
          0.3397, -0.3619,  0.3563, -1.3732,  0.6811, -0.3405, -0.1920, -0.2058],
        [-0.9904,  0.1

The parameters used for producing keys,queries and values are updated during back propagation while training.

In [50]:
q[0] # query for 1st batch

tensor([[ 1.0195,  0.0180,  0.3499, -0.1077, -0.1675,  0.9082,  0.3237,  0.3342,
          0.6764,  0.5757, -0.2923, -0.1950, -0.1491,  0.3595, -0.0443, -0.5720],
        [ 0.2280,  0.3105,  0.2529,  0.6757, -0.0604,  0.5528,  0.1813,  0.7440,
         -0.2352, -0.7467,  0.7603, -0.4731,  0.9524, -0.2704, -0.9648, -0.3232],
        [-0.5946,  0.0298,  0.0676, -1.5549,  0.2101,  0.3188, -0.4135, -0.6545,
          0.7547, -0.2539, -0.6663,  0.5993, -1.2053, -1.0629, -0.4573, -0.0328],
        [-0.3424,  0.5093,  0.9616, -0.5442, -1.0766,  0.5237,  0.0549, -0.4219,
         -0.3021, -1.3860,  0.0161, -0.6141,  0.5594,  0.2768, -0.1585,  0.1855],
        [-0.0585, -0.2958,  0.4430,  0.2243,  0.3053,  1.0938, -0.3049,  0.5256,
          0.4267,  0.5015, -0.0285, -0.0875,  0.2724,  0.1113, -0.3839, -1.0527],
        [ 0.0452,  0.2018,  0.3754,  1.4890,  0.5628,  1.0335,  0.1353, -0.6198,
          0.4369, -0.3828, -1.1007,  0.2396, -0.3933, -0.5930,  0.9824,  0.4434],
        [-0.6606,  0.1

In [51]:
wei[0]

tensor([[-0.0930,  0.4241, -0.1920, -0.2604, -0.2047,  0.3298, -0.1850,  0.1063],
        [-0.1317, -0.0041,  0.3544, -0.5092,  0.3235,  0.6735, -0.4925, -0.3394],
        [-0.1386,  0.1132, -0.4332, -0.1018, -0.4526, -0.4877,  0.2924,  0.1479],
        [-0.3793, -0.4448, -0.0904, -0.3852, -0.4691,  0.0264,  0.1855,  0.1810],
        [-0.2314,  0.3281,  0.2602, -0.1761, -0.1294,  0.2328, -0.0882, -0.3046],
        [ 0.1535,  0.4400,  0.1190,  0.1333, -0.1915, -0.0126,  0.3199,  0.1040],
        [-0.1539, -0.0132, -0.2436,  0.0049, -0.1258, -0.5058,  0.3951,  0.2959],
        [-0.2712,  0.4028,  0.0933,  0.3936, -0.3119,  0.0054,  0.2060,  0.2647]],
       grad_fn=<SelectBackward0>)

This is the attention matrix that shows the affinity level of one token with respect to other. Here, the past tokens are also communicating with the future tokens which happens most during the encoder phase in tranformer.Here, the 1st token, for instance, is communicating with the 8th token and the value of 0.1063 at (1,8) depicts the attention value. We can see, the matrix isn't properly diffused.

In [52]:
wei = wei.masked_fill(tril == 0, float('-inf')) # (B, T, T)

In [53]:
wei[0]

tensor([[-0.0930,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1317, -0.0041,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1386,  0.1132, -0.4332,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.3793, -0.4448, -0.0904, -0.3852,    -inf,    -inf,    -inf,    -inf],
        [-0.2314,  0.3281,  0.2602, -0.1761, -0.1294,    -inf,    -inf,    -inf],
        [ 0.1535,  0.4400,  0.1190,  0.1333, -0.1915, -0.0126,    -inf,    -inf],
        [-0.1539, -0.0132, -0.2436,  0.0049, -0.1258, -0.5058,  0.3951,    -inf],
        [-0.2712,  0.4028,  0.0933,  0.3936, -0.3119,  0.0054,  0.2060,  0.2647]],
       grad_fn=<SelectBackward0>)

During the language modeling and text generation process, we don't want future tokens to communicate with the past tokens. We only want an interaction between the past tokens and the present tokens. Hence, we are masking the future values as shown by 'inf'. 

In [54]:
wei = F.softmax(wei, dim=-1) # (B, T, T)
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4681, 0.5319, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3299, 0.4244, 0.2457, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2344, 0.2196, 0.3129, 0.2330, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1527, 0.2672, 0.2496, 0.1614, 0.1691, 0.0000, 0.0000, 0.0000],
        [0.1714, 0.2283, 0.1656, 0.1680, 0.1214, 0.1452, 0.0000, 0.0000],
        [0.1299, 0.1495, 0.1187, 0.1522, 0.1336, 0.0913, 0.2248, 0.0000],
        [0.0837, 0.1643, 0.1205, 0.1628, 0.0804, 0.1104, 0.1349, 0.1431]],
       grad_fn=<SelectBackward0>)

For the first batch, this is the matrix multiplication/dot product of keys and query which tells us how close are the relationship between tokens. Every values in that matrix shows the affinity level between different tokens. As the key of that token multiplies with the query of the preceding tokens, the one with higher value signifies the affinity level/ interest level between them.

For example:

The number '0.1349' is the multiplication of key of 8th token with the query of 7th token. So, this number tells us the affinity the 8th token has with the 7th token. By the same logic, we can also conclude therefore that the 8th token has highest affinity with the 2nd token in the first batch. This is just the result of a single self-attention head. Remember, there are more than 4 self-attention heads in transformer. Each self-attention mechanism works in parallel. So, with these mechanism, we might infer several affinity or interesting things for that 8th token through the great use of mult-attention-heads. 


Similarly, through this attention mechanism, for the 8th token, we now know that 8 percent info of the 1st token, 16.43% of the 2nd token, 12% of the 3rd token, 16% of the 4th token, 8% of the fifth token, 11% of the 6th token, 13% of the 7th token and 14% of the current token(i.e 8th token)...all these informations are aggregated for the current token (which is 8th token in this case). Hence, the aggregations of this information is achieved by matrix multiplication of value(content) against wei(attention scores).

In [59]:
v = value(x) # (B,T,hs)

'v' means value, which is one of the three tridents of self-attention mechanism. Value simply contains the CONTENT of that token.

In [60]:
out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
out[0]

tensor([[ 0.0734,  0.4547, -0.4117,  0.2293, -0.8554,  0.0896, -0.4814,  0.4628,
          0.8049, -0.4331, -0.2530, -0.4248, -0.2936, -0.4427, -0.4643,  0.6501],
        [-0.0968,  0.3969, -0.0825,  0.4059, -1.0984,  0.1415, -0.7913,  0.0784,
          0.6317,  0.0491,  0.1363, -0.5873, -0.1390, -0.1602,  0.1914,  0.8521],
        [-0.3016,  0.2229,  0.0852,  0.4630, -0.7486, -0.0449, -0.6494,  0.0214,
          0.4461,  0.1686,  0.0332, -0.3992, -0.1913, -0.2589,  0.0537,  0.5827],
        [-0.2630,  0.2296,  0.2290,  0.4954, -0.4615, -0.1865, -0.3838,  0.0033,
          0.1775, -0.3129, -0.1409, -0.0233, -0.3206, -0.3656, -0.0993,  0.1139],
        [-0.2569,  0.1265,  0.2566,  0.3564, -0.7354, -0.1099, -0.3854, -0.0496,
          0.1845, -0.0595, -0.1291, -0.0714, -0.2054, -0.3579, -0.1348,  0.1172],
        [-0.1120,  0.1607,  0.0206,  0.3914, -0.7536, -0.1724, -0.4604, -0.0232,
          0.0911, -0.2189, -0.0069, -0.0745, -0.2770, -0.3293, -0.1000,  0.0358],
        [-0.1445,  0.1